In [3]:
import pandas as pd
import os
import numpy
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor

In [4]:
pd.set_option('display.max_columns', None)
data = pd.read_csv("/Users/ajinkya.parkar@ibm.com/Documents/deep/House price prediction/data/train.csv")
data_test = pd.read_csv("/Users/ajinkya.parkar@ibm.com/Documents/deep/House price prediction/data/test.csv")

In [5]:
for missingCols in data.columns[data.isnull().any()]:
    print(missingCols)

LotFrontage
Alley
MasVnrType
MasVnrArea
BsmtQual
BsmtCond
BsmtExposure
BsmtFinType1
BsmtFinType2
Electrical
FireplaceQu
GarageType
GarageYrBlt
GarageFinish
GarageQual
GarageCond
PoolQC
Fence
MiscFeature


# drop corelated columns

In [6]:
#data[["LotFrontage","Alley","MasVnrType","MasVnrArea","BsmtQual","BsmtCond","BsmtExposure","BsmtFinType1","BsmtFinType2","Electrical","FireplaceQu","GarageType","GarageYrBlt","GarageFinish","GarageQual","GarageCond","PoolQC","Fence","MiscFeature","SalePrice"]]

data = data.drop(columns=["Alley","PoolQC","Fence","MiscFeature","GarageCars","Exterior1st","GrLivArea"])



# fill na values

In [7]:
dataIndex = data.fillna("NA")
data_testIndex = data_test.fillna("NA")


In [8]:
data = data.fillna("NA").drop(columns=["Id"])
data_test = data_test.fillna("NA").drop(columns=["Id"])


# Convert to category codes for columns with datatype object

In [9]:
for col_name in data.columns:
    if(data[col_name].dtype == 'object'):
        data[col_name]= data[col_name].astype('category')
        data[col_name] = data[col_name].cat.codes
        
for col_name in data_test.columns:
    if(data_test[col_name].dtype == 'object'):
        data_test[col_name]= data_test[col_name].astype('category')
        data_test[col_name] = data_test[col_name].cat.codes        

In [10]:
scaler = MinMaxScaler(feature_range=(0, 1))
#scaled_data = scaler.fit_transform(data)
#scaled_dataTest = scaler.fit_transform(data_test)

In [11]:
from sklearn.feature_selection import VarianceThreshold
sel = VarianceThreshold(threshold=(.8 * (1 - .8)))




# Chi square test

In [12]:
arrayTrain = sel.fit_transform(data.values) #data.drop(columns=["ExterQual","BsmtQual","KitchenQual"]).values
X = arrayTrain[:,0:arrayTrain.shape[1]-1]
Y = arrayTrain[:,arrayTrain.shape[1]-1]

arrayTest = data_test.values #data_test.drop(columns=["ExterQual","BsmtQual","KitchenQual"]).values
test_X = arrayTest[:,0:arrayTest.shape[1]-1]
#test_y = array[:,array.shape[1]-1]


In [13]:
def scaleData(X_data,y_data,scaler=MinMaxScaler(feature_range=(0, 1))):
    #scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(X_data)
    scaled_dataOP = scaler.fit_transform(y_data)
    return scaled_data,scaled_dataOP

In [14]:
from sklearn.metrics import mean_squared_error

In [15]:
SelectKBest(score_func=chi2,k= 10)

SelectKBest(k=10, score_func=<function chi2 at 0x113d5b1e0>)

In [30]:
X.shape

(1460, 64)

In [31]:
Y.shape

(1460,)

In [32]:
test = SelectKBest(score_func=chi2,k= arrayTrain.shape[1]-1)
test.fit(X, Y)

SelectKBest(k=64, score_func=<function chi2 at 0x113d5b1e0>)

In [65]:
test = SelectKBest(score_func=chi2,k= arrayTrain.shape[1]-1)
fitChi = test.fit(X, Y)
dfscores = pd.DataFrame(fitChi.scores_)
dfcolumns = pd.DataFrame(data.columns)
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Specs','Score']
bestFeatureCombinationCount = 0
RMS = 1
for i in range(5,len(data.columns)-1):
    
    topFeatures = featureScores.nlargest(i,'Score').pivot(index='Score', columns='Specs', values='Score')
    intersection_cols = data.columns & topFeatures.columns
    scaler = MinMaxScaler(feature_range=(0, 1))
    train_X,train_Y = scaleData(data[intersection_cols],data.values[:,data.shape[1]-1].reshape(1460,1),scaler)
    numpy.set_printoptions(precision=3)
    reg = RandomForestRegressor(n_estimators = 1000, random_state = 42).fit(train_X, train_Y)
    print(mean_squared_error(reg.predict(train_X),train_Y))
    if(mean_squared_error(reg.predict(train_X),train_Y) < RMS):
        bestFeatureCombinationCount = i
        RMS = mean_squared_error(reg.predict(train_X),train_Y)
        print("i = ",i)


/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  app.launch_new_instance()


0.000812987011317
i =  5


/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  app.launch_new_instance()


0.000803903932274
i =  6


/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  app.launch_new_instance()


...

0.000232572351345


/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  app.launch_new_instance()


0.000230570420225


/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  app.launch_new_instance()


0.000231089662238


In [67]:
bestFeatureCombinationCount

68

# select best features with minimum rms error

In [170]:
from sklearn.datasets import make_friedman1
from sklearn.feature_selection import RFE
from sklearn.svm import SVR
estimator = SVR(kernel="linear")
#RFE = RFE(estimator, 70, step=1)



In [164]:
#selectorModel = RFE.fit_transform(X, Y)


In [157]:
reg = RandomForestRegressor(n_estimators = 1000, random_state = 42).fit(selectorModel, train_Y)
print(mean_squared_error(reg.predict(selectorModel),train_Y))

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


0.000230506227793


In [133]:
from sklearn.datasets import make_friedman1
from sklearn.feature_selection import RFE
from sklearn.svm import SVR
test = make_friedman1(n_samples=50, n_features=10, random_state=0)
fitChi = test.fit_transform(X, Y)
reg = RandomForestRegressor(n_estimators = 1000, random_state = 42).fit(fitChi, train_Y)
print(mean_squared_error(reg.predict(fitChi),train_Y))


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


0.000472927540198


In [171]:
for i in range(50,len(data.columns)-1):
    print(i)
    RFEModel = RFE(estimator, i, step=1)
    transformedData = RFEModel.fit_transform(X, Y)
    reg = RandomForestRegressor(n_estimators = 1000, random_state = 42).fit(transformedData, train_Y)
    
    print(mean_squared_error(reg.predict(transformedData),train_Y))

50


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


0.00023388027808
51


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


0.000232453467628
52


...

0.000231497330103
70


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


0.000230506227793
71


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


0.000230266465766


In [395]:

topFeatures = featureScores.nlargest(60,'Score').pivot(index='Score', columns='Specs', values='Score')
intersection_cols = data.columns & topFeatures.columns
scaler = MinMaxScaler(feature_range=(0, 1))
train_X,train_Y = scaleData(data[intersection_cols],data.values[:,data.shape[1]-1].reshape(data.shape[0],1),scaler)


/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [396]:
train_X.shape

(1460, 60)

# Train Random forest regression model

In [398]:
from sklearn.model_selection import train_test_split

In [399]:
X_train, X_test, y_train, y_test = train_test_split(train_X, train_Y, test_size=0.20, random_state=42)

In [400]:
from sklearn.ensemble import RandomForestRegressor
RDFreg = RandomForestRegressor(n_estimators = 300, random_state = 42,max_depth=6,criterion = "mse").fit(X_train, y_train)
Linreg = LinearRegression().fit(train_X, train_Y)


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


In [401]:
#scaler.inverse_transform([reg.predict(X_train)])

In [402]:
mean_squared_error(Linreg.predict(X_train),y_train)

0.001960402788214681

In [403]:
mean_squared_error(RDFreg.predict(X_train),y_train)

0.0008402972808603307

In [404]:
mean_squared_error(Linreg.predict(X_test),y_test)

0.0019911472188535498

In [405]:
mean_squared_error(RDFreg.predict(X_test),y_test)

0.0019488012920608857

In [240]:
'''from sklearn import metrics
max_depths = np.linspace(1, 32, 32, endpoint=True)
train_results = []
test_results = []
for max_depth in max_depths:
   rf = RandomForestRegressor(max_depth=max_depth, n_jobs=-1)
   rf.fit(X_train, y_train)
   train_pred = rf.predict(X_train)
   false_positive_rate, true_positive_rate, thresholds = metrics.roc_curve(y_train, train_pred)
   roc_auc = auc(false_positive_rate, true_positive_rate)
   train_results.append(roc_auc)
   y_pred = rf.predict(X_test)
   false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
   roc_auc = auc(false_positive_rate, true_positive_rate)
   test_results.append(roc_auc)
from matplotlib.legend_handler import HandlerLine2D
line1, = plt.plot(max_depths, train_results, "b", label="Train AUC")
line2, = plt.plot(max_depths, test_results, "r", label="Test AUC")
plt.legend(handler_map={line1: HandlerLine2D(numpoints=2)})
plt.ylabel("AUC score")
plt.xlabel("Tree depth")
plt.show()
'''

'from sklearn import metrics\nmax_depths = np.linspace(1, 32, 32, endpoint=True)\ntrain_results = []\ntest_results = []\nfor max_depth in max_depths:\n   rf = RandomForestRegressor(max_depth=max_depth, n_jobs=-1)\n   rf.fit(X_train, y_train)\n   train_pred = rf.predict(X_train)\n   false_positive_rate, true_positive_rate, thresholds = metrics.roc_curve(y_train, train_pred)\n   roc_auc = auc(false_positive_rate, true_positive_rate)\n   train_results.append(roc_auc)\n   y_pred = rf.predict(X_test)\n   false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)\n   roc_auc = auc(false_positive_rate, true_positive_rate)\n   test_results.append(roc_auc)\nfrom matplotlib.legend_handler import HandlerLine2D\nline1, = plt.plot(max_depths, train_results, "b", label="Train AUC")\nline2, = plt.plot(max_depths, test_results, "r", label="Test AUC")\nplt.legend(handler_map={line1: HandlerLine2D(numpoints=2)})\nplt.ylabel("AUC score")\nplt.xlabel("Tree depth")\nplt.show()\n'

# predict and inverse the results for training set

In [131]:
scaler.inverse_transform([reg.predict(train_X)])

array([[ 210490.013,  178834.185,  217498.757, ...,  261054.758,
         144348.027,  153264.4  ]])

In [132]:
scaler.inverse_transform(train_Y)

array([[ 208500.],
       [ 181500.],
       [ 223500.],
       ..., 
       [ 266500.],
       [ 142125.],
       [ 147500.]])

# test data prediction

In [74]:
topFeatures = featureScores.nlargest(bestFeatureCombinationCount,'Score').pivot(index='Score', columns='Specs', values='Score')
intersection_cols = data_test.columns & topFeatures.columns
scalerTest = MinMaxScaler(feature_range=(0, 1))
test_X,test_Y = scaleData(data_test[intersection_cols],data_test.values[:,data_test.shape[1]-1].reshape(data_test.shape[0],1),scalerTest)


/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [75]:
test_X.shape

(1459, 68)

In [76]:
reg.predict(test_X)

array([ 0.158,  0.167,  0.2  , ...,  0.199,  0.133,  0.342])

In [77]:
output = scaler.inverse_transform([reg.predict(test_X)])
output

array([[ 148654.146,  154974.227,  178896.108, ...,  178268.332,
         130680.244,  281381.748]])

In [78]:
output[0,:]

array([ 148654.146,  154974.227,  178896.108, ...,  178268.332,
        130680.244,  281381.748])

In [172]:
dataset = pd.DataFrame({'SalePrice':output[0,:]})


In [80]:
pd.concat([data_testIndex["Id"],dataset ], axis=1, join_axes=[dataset.index]).set_index(['Id']).to_csv(r'test_submission_V5.csv')
